In [28]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense
from keras.utils import to_categorical 

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

df = pd.read_csv('2016-forecast-cnn.csv')
df = df.drop(['Unnamed: 0','No','NIM'],axis=1)
df.head()

,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,Strukdat,SE,PO,SCM,EA,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi,Lulus Tepat Waktu
0,4.0,3.5,4.0,3.5,4.0,3.5,3.5,3.5,3.5,4.0,3.5,3.5,4.0,3.5,3.5,4.0,3.5,3.5,3.5,1
1,4.0,3.5,3.5,3.5,3.5,3.5,3.5,4.0,3.5,3.5,3.5,4.0,3.5,3.5,3.0,3.5,4.0,3.5,3.5,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0
3,4.0,4.0,3.5,3.5,4.0,3.0,4.0,3.5,3.0,3.5,3.0,3.5,3.5,4.0,3.5,3.5,3.5,2.5,3.5,1
4,3.5,3.0,3.5,2.0,2.0,2.5,4.0,3.5,2.0,3.5,2.5,2.5,3.5,3.5,2.5,2.0,1.0,4.0,4.0,1


In [29]:
df2 = pd.read_csv('2016 data v5.csv')
df2.head()

,No,NIM,OOP,RPB,RPL,PI,APSI,WEB,Statistik,Matdis,...,EA,Basdat,Manjarkom,Sisop,MSDM,Desjar,KSI,Pilihan 1,Pilihan 2,Peminatan
0,1,1202160002,4.0,3.5,0.0,4.0,3.5,4.0,3.5,3.5,...,4.0,3.5,3.5,4.0,3.5,3.5,0.0,ERP,EDM,ERP
1,2,1202160009,4.0,3.5,0.0,3.5,3.5,3.5,3.5,3.5,...,3.5,3.5,3.0,3.5,4.0,3.5,0.0,ERP,EDM,ERP
2,3,1202160010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0,NaN
3,4,1202160011,4.0,4.0,0.0,3.5,3.5,4.0,3.0,4.0,...,3.5,4.0,3.5,3.5,3.5,2.5,0.0,ERP,EDM,ERP
4,5,1202160013,3.5,3.0,0.0,3.5,2.0,2.0,2.5,4.0,...,3.5,3.5,2.5,2.0,1.0,4.0,0.0,EA,EDM,EDM


In [30]:
df['Pilihan 1'] = df2['Pilihan 1']
df['Pilihan 2'] = df2['Pilihan 2']
df['Peminatan'] = df2['Peminatan']
df.head()

,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,Strukdat,SE,...,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi,Lulus Tepat Waktu,Pilihan 1,Pilihan 2,Peminatan
0,4.0,3.5,4.0,3.5,4.0,3.5,3.5,3.5,3.5,4.0,...,3.5,3.5,4.0,3.5,3.5,3.5,1,ERP,EDM,ERP
1,4.0,3.5,3.5,3.5,3.5,3.5,3.5,4.0,3.5,3.5,...,3.5,3.0,3.5,4.0,3.5,3.5,1,ERP,EDM,ERP
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.5,0,NaN,0,NaN
3,4.0,4.0,3.5,3.5,4.0,3.0,4.0,3.5,3.0,3.5,...,4.0,3.5,3.5,3.5,2.5,3.5,1,ERP,EDM,ERP
4,3.5,3.0,3.5,2.0,2.0,2.5,4.0,3.5,2.0,3.5,...,3.5,2.5,2.0,1.0,4.0,4.0,1,EA,EDM,EDM


In [31]:
new_class = [0] * len(df['Peminatan'])
for index, row in df.iterrows() :
    if row["Peminatan"] == "TECHNO" :
        new_class[index] = 1
    elif row['Peminatan'] == "EAD" :
        new_class[index] = 2
    elif row['Peminatan'] == "EDM" :
        new_class[index] = 3
    elif row['Peminatan'] == "ERP" :
        new_class[index] = 4
    elif row['Peminatan'] == "EA" :
        new_class[index] = 5
    elif row['Peminatan'] == "EIM" :
        new_class[index] = 6
    elif row['Peminatan'] == "ISM" :
        new_class[index] = 7
    else :
        new_class[index] = 0
df["Peminatan"] = new_class
df.head()

,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,Strukdat,SE,...,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi,Lulus Tepat Waktu,Pilihan 1,Pilihan 2,Peminatan
0,4.0,3.5,4.0,3.5,4.0,3.5,3.5,3.5,3.5,4.0,...,3.5,3.5,4.0,3.5,3.5,3.5,1,ERP,EDM,4
1,4.0,3.5,3.5,3.5,3.5,3.5,3.5,4.0,3.5,3.5,...,3.5,3.0,3.5,4.0,3.5,3.5,1,ERP,EDM,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.5,0,NaN,0,0
3,4.0,4.0,3.5,3.5,4.0,3.0,4.0,3.5,3.0,3.5,...,4.0,3.5,3.5,3.5,2.5,3.5,1,ERP,EDM,4
4,3.5,3.0,3.5,2.0,2.0,2.5,4.0,3.5,2.0,3.5,...,3.5,2.5,2.0,1.0,4.0,4.0,1,EA,EDM,3


In [32]:
new_class = [0] * len(df['Pilihan 1'])
for index, row in df.iterrows() :
    if row["Pilihan 1"] == "TECHNO" :
        new_class[index] = 1
    elif row['Pilihan 1'] == "EAD" :
        new_class[index] = 2
    elif row['Pilihan 1'] == "EDM" :
        new_class[index] = 3
    elif row['Pilihan 1'] == "ERP" :
        new_class[index] = 4
    elif row['Pilihan 1'] == "EA" :
        new_class[index] = 5
    elif row['Pilihan 1'] == "EIM" :
        new_class[index] = 6
    elif row['Pilihan 1'] == "ISM" :
        new_class[index] = 7
    else :
        new_class[index] = 0
df["Pilihan 1"] = new_class
df.head()

,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,Strukdat,SE,...,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi,Lulus Tepat Waktu,Pilihan 1,Pilihan 2,Peminatan
0,4.0,3.5,4.0,3.5,4.0,3.5,3.5,3.5,3.5,4.0,...,3.5,3.5,4.0,3.5,3.5,3.5,1,4,EDM,4
1,4.0,3.5,3.5,3.5,3.5,3.5,3.5,4.0,3.5,3.5,...,3.5,3.0,3.5,4.0,3.5,3.5,1,4,EDM,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.5,0,0,0,0
3,4.0,4.0,3.5,3.5,4.0,3.0,4.0,3.5,3.0,3.5,...,4.0,3.5,3.5,3.5,2.5,3.5,1,4,EDM,4
4,3.5,3.0,3.5,2.0,2.0,2.5,4.0,3.5,2.0,3.5,...,3.5,2.5,2.0,1.0,4.0,4.0,1,5,EDM,3


In [33]:
new_class = [0] * len(df['Pilihan 2'])
for index, row in df.iterrows() :
    if row["Pilihan 2"] == "TECHNO" :
        new_class[index] = 1
    elif row['Pilihan 2'] == "EAD" :
        new_class[index] = 2
    elif row['Pilihan 2'] == "EDM" :
        new_class[index] = 3
    elif row['Pilihan 2'] == "ERP" :
        new_class[index] = 4
    elif row['Pilihan 2'] == "EA" :
        new_class[index] = 5
    elif row['Pilihan 2'] == "EIM" :
        new_class[index] = 6
    elif row['Pilihan 2'] == "ISM" :
        new_class[index] = 7
    else :
        new_class[index] = 0
df["Pilihan 2"] = new_class
df.head()

,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,Strukdat,SE,...,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi,Lulus Tepat Waktu,Pilihan 1,Pilihan 2,Peminatan
0,4.0,3.5,4.0,3.5,4.0,3.5,3.5,3.5,3.5,4.0,...,3.5,3.5,4.0,3.5,3.5,3.5,1,4,3,4
1,4.0,3.5,3.5,3.5,3.5,3.5,3.5,4.0,3.5,3.5,...,3.5,3.0,3.5,4.0,3.5,3.5,1,4,3,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.5,0,0,0,0
3,4.0,4.0,3.5,3.5,4.0,3.0,4.0,3.5,3.0,3.5,...,4.0,3.5,3.5,3.5,2.5,3.5,1,4,3,4
4,3.5,3.0,3.5,2.0,2.0,2.5,4.0,3.5,2.0,3.5,...,3.5,2.5,2.0,1.0,4.0,4.0,1,5,3,3


In [40]:
mainData = df
mainData = mainData.drop(['Lulus Tepat Waktu'],axis=1)
target = ["Peminatan"]
predictors = list(set(list(mainData.columns))-set(target))
mainData[predictors] = mainData[predictors]/mainData[predictors].max()
mainData[predictors].head()

,RPB,OOP,MSDM,Basdat,PI,EA,Manjarkom,Manprosi,APSI,SE,...,SCM,PO,Statistik,Matdis,Desjar,Pilihan 2,Alpro,WEB,Pilihan 1,Sisop
0,0.875,1.000,0.875,0.875,1.000,1.000,0.875,0.875,0.875,1.000,...,0.875,0.875,0.875,0.875,0.875,0.428571,0.875,1.000,0.571429,1.000
1,0.875,1.000,1.000,0.875,0.875,0.875,0.750,0.875,0.875,0.875,...,1.000,0.875,0.875,0.875,0.875,0.428571,1.000,0.875,0.571429,0.875
2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.875,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000000,0.000,0.000,0.000000,0.000
3,1.000,1.000,0.875,1.000,0.875,0.875,0.875,0.875,0.875,0.875,...,0.875,0.750,0.750,1.000,0.625,0.428571,0.875,1.000,0.571429,0.875
4,0.750,0.875,0.250,0.875,0.875,0.875,0.625,1.000,0.500,0.875,...,0.625,0.625,0.625,1.000,1.000,0.428571,0.875,0.500,0.714286,0.500


In [43]:
X = mainData[target]
y = mainData[predictors]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=40)
print(X_train.shape)
print(y_train.shape)
print(X_test.head())
print(y_test.shape)

(241, 1)
(241, 21)
     Peminatan
139          0
56           6
301          1
187          7
312          0
(162, 21)


In [45]:
X_trains = to_categorical(X_train)
X_tests = to_categorical(X_test)

print(X_trains.shape)

(241, 8)


In [47]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=19))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])